# Performance comparison of sequential vs parallel bubble sort in mpi4py.

## Bubble Sort (Sequential)

In [52]:
def bubbleSort(arr):
    n = len(arr)
    for i in range(n-1):
        swapped = False
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                swapped = True
                arr[j], arr[j+1] = arr[j+1], arr[j]
        
        if not swapped: 
            return

- Time Complexity: O(n^2)
- Space Complexity: O(1)

## Populating 3 arrays with 1000, 10000 and 100000 random integers respectively:

In [53]:
import numpy as np
# generate random integer values
from numpy.random import seed
from numpy.random import randint
# seed random number generator
seed(1)

array_100 = randint(1, 1000, 100)
array_1000 = randint(1, 1000, 1000)
array_10000 = randint(1, 1000, 10000)

print("length of 1st array = ", len(array_100))
print("length of 2nd array = ", len(array_1000))
print("length of 3rd array = ", len(array_10000))



length of 1st array =  100
length of 2nd array =  1000
length of 3rd array =  10000


### Time taken to sort an array with 100 elements:

In [54]:
import time 
begin = time.time()

bubbleSort(array_100)

end = time.time()

print(f"Total runtime of the program to sort 100 array elements is {end - begin} seconds")

Total runtime of the program to sort 100 array elements is 0.004719972610473633 seconds


### Time taken to sort an array with 1000 elements:

In [55]:
begin = time.time()

bubbleSort(array_1000)

end = time.time()

print(f"Total runtime of the program to sort 1000 array elements is {end - begin} seconds")

Total runtime of the program to sort 1000 array elements is 0.3431673049926758 seconds


### Time taken to sort an array with 10000 elements:

In [56]:
begin = time.time()

bubbleSort(array_10000)

end = time.time()

print(f"Total runtime of the program to sort 10000 array elements is {end - begin} seconds")

KeyboardInterrupt: 

## Parallel Bubble Sort (using mpi4py):

In [58]:
from mpi4py import MPI
import numpy as np

def parallel_bubble_sort(arr, comm):
    size = comm.Get_size()
    rank = comm.Get_rank()
    local_size = len(arr)
    
    # Scatter data among processes
    local_arr = np.empty(local_size, dtype=int)
    comm.Scatter(arr, local_arr, root=0)
    
    # Perform local bubble sort
    for i in range(local_size):
        for j in range(i + 1, local_size):
            if local_arr[i] > local_arr[j]:
                local_arr[i], local_arr[j] = local_arr[j], local_arr[i]
    
    # Gather sorted sublists from all processes
    sorted_arr = np.empty_like(arr)
    comm.Gather(local_arr, sorted_arr, root=0)
    
    # Merge sorted sublists
    if rank == 0:
        for i in range(1, size):
            start = i * local_size
            end = min((i + 1) * local_size, len(arr))
            sorted_arr[start:end] = comm.recv(source=i, tag=i)
    else:
        comm.send(local_arr, dest=0, tag=rank)
    
    return sorted_arr


ImportError: libmpi.so.20: cannot open shared object file: No such file or directory